<h1>Исследование надёжности заёмщиков</h1>

<h2>Описание проекта</h2>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте таблицу и изучите общую информацию о данных ##

Импортируем библиотекиу pandas для обработки и анализа табличных данных, а также библиотеку pymystem3 для проведения лемматизации.

In [21]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

Читаем csv-файл и выводим на данные.

In [22]:
df = pd.read_csv('data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод ###
* Файл содержит таблицу, состоящую 21 525 строк и 12 колонок.
* В колонках days_employed и total_income есть не все данные (по 19 351 строке).
* В колонках days_employed и total_income указан некорректный тип данных (float64).
* В колонках children и days_employed есть ошибки в данных.
* Некоторые колонки нужно переименовать для лучшего понимания данных.

## Шаг 2. Предобработка данных ##

### Изменение названий столбцов и обработка пропусков ###

Выводим текущие названия столбцов.

In [23]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Корректируем названия столбцов.

In [24]:
df.rename(columns={'children': 'children_number', 'dob_years': 'age_years', 'education': 'education_level', 'education_id': 'education_level_id', 'purpose': 'loan_purpose'}, inplace=True)
df.columns

Index(['children_number', 'days_employed', 'age_years', 'education_level',
       'education_level_id', 'family_status', 'family_status_id', 'gender',
       'income_type', 'debt', 'total_income', 'loan_purpose'],
      dtype='object')

### Обработка пропусков ###

Считаем доли пропусков.

In [25]:
df.isna().mean()

children_number       0.000000
days_employed         0.100999
age_years             0.000000
education_level       0.000000
education_level_id    0.000000
family_status         0.000000
family_status_id      0.000000
gender                0.000000
income_type           0.000000
debt                  0.000000
total_income          0.100999
loan_purpose          0.000000
dtype: float64

Доля пропусков в колонках days_employed и total_income одинаковая — 10%. Причинами появления пропусков могут быть технические проблемы при выгрузке из разных баз данных.

### Заполненияем пропуски ###

days_employed: данные по этому столбцу не фигурируют в запросе Заказчика, поэтому пропуски заменяем нулями.

In [26]:
df['days_employed'] = df['days_employed'].fillna(0)

total_income заполним медианным значением в соответствии с типом занятости. Для этого сделаем:
— расчет медианного значения по `income_type` и сохранение его в `med_table`

In [27]:
med_table = df.groupby('income_type')['total_income'].median()

— подставновку рассчитанного медианного значения в пропущенные ячейки

In [28]:
for income_type in med_table.index:
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type), 'total_income'] = med_table.loc[income_type]

— проверяем наличие пропусков

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children_number     21525 non-null  int64  
 1   days_employed       21525 non-null  float64
 2   age_years           21525 non-null  int64  
 3   education_level     21525 non-null  object 
 4   education_level_id  21525 non-null  int64  
 5   family_status       21525 non-null  object 
 6   family_status_id    21525 non-null  int64  
 7   gender              21525 non-null  object 
 8   income_type         21525 non-null  object 
 9   debt                21525 non-null  int64  
 10  total_income        21525 non-null  float64
 11  loan_purpose        21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [30]:
med_table

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

### Вывод ###
* Скорректировано название колонок.
* Заполнены все пропуски в данных.

### Замена типа данных ###

Заменяем в колонках days_employed и total_income тип данных с float64 на int32.

In [31]:
for col in ['days_employed', 'total_income']:
    df[col] = df[col].astype('int')
df.dtypes

children_number        int64
days_employed          int32
age_years              int64
education_level       object
education_level_id     int64
family_status         object
family_status_id       int64
gender                object
income_type           object
debt                   int64
total_income           int32
loan_purpose          object
dtype: object

### Вывод ###
* Тип данных изменен. Метод для их изменения был выбран в соответствии с начальным типом данных — float64.

### Обработка дубликатов ###

Для более эффективного поиска дубликатов приведем данные в таблице к нижнему регистру с применением конструкции try-except, чтобы исключить возможные ошибки отсутствия атрибута str.lower в колонках таблицы — AttributeError.

In [32]:
for col in df.columns:
    try:
        df[col] = df[col].str.lower()
    except AttributeError:
        pass

Считаем количество дубликатов в таблице.

In [33]:
df.duplicated().sum()

71

Удаляем дубликаты с помощью drop_duplicates, оставляя первую строку из перечня дублей.

In [34]:
df = df.drop_duplicates()

In [35]:
df.duplicated().sum()

0

In [36]:
df.head()

,children_number,days_employed,age_years,education_level,education_level_id,family_status,family_status_id,gender,income_type,debt,total_income,loan_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу


### Вывод ###
* В таблице был 71 дубликат. После их выявления они были удалены. Возможные причины появления дубликатов — технические проблемы при выгрузке из разных баз данных.

### Категоризация данных ###

При работе с набором данных были созданы и использованы следующие «словари»:

* columns — для переименования названий колонок;
* med_table — для подставновки рассчитанного медианного значения в пропущенные ячейки колонки total_income;
* {-1: 1, 20: 2} — для корректировки значений в колонке children_number.

In [37]:
df['children_number'] = df['children_number'].replace({-1 : 1, 20 : 2})
df.children_number.value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children_number, dtype: int64

## Шаг 3. Ответьте на вопросы ## 

* Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
children_debt = df.pivot_table(index = 'children_number', values = 'debt', aggfunc=['count','sum'])
children_debt['%'] = round((children_debt[('sum', 'debt')] / children_debt[('count', 'debt')])*100,2)
children_debt.columns = ['Общее количество заемщиков', 'Количество просрочников','%']
children_debt

,Общее количество заемщиков,Количество просрочников,%
children_number,,,
0,14091,1063,7.54
1,4855,445,9.17
2,2128,202,9.49
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


### Вывод ###
Несмотря на максимальное количество клиентов с просроченными кредитами у которых нет детей, доля их в сравнении с иными категориями клиентов меньшая и составляет 7,54%, в то время как доли просроченных кредитов у клиентов имеющих от 1 до 4 детей — от 8,18% до 9,76%. В силу малого количества попавших в выборку клиентов с 5-ю детьми, данные по ним можно не учитывать.

* Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
family_status_debt = df.pivot_table(index = 'family_status', values = 'debt',aggfunc = ['count', 'sum'])
family_status_debt['%'] = round((family_status_debt[('sum', 'debt')] / family_status_debt[('count', 'debt')])*100,2)
family_status_debt.columns = ['Общее количество заемщиков', 'Количество просрочников','%']
family_status_debt.sort_values(by = '%', ascending = False)

,Общее количество заемщиков,Количество просрочников,%
family_status,,,
не женат / не замужем,2810,274,9.75
гражданский брак,4151,388,9.35
женат / замужем,12339,931,7.55
в разводе,1195,85,7.11
вдовец / вдова,959,63,6.57


### Вывод ###
В большей степени просрочку допускают заемщики, не женатые / не замужние — 9,75% от общего количества всех не женатых / не замужних. Также от них не сильно отстают те, кто живет в гражданском браке — 9,35% от общего числа состоящих в гражданском браке.

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
total_income_debt = pd.DataFrame(df, columns = ['total_income', 'debt'])
total_income_debt['income_groups'] = pd.cut(total_income_debt['total_income'],[0,50000,100000,150000,200000,np.inf], labels = ["до 50 тыс. руб.", "50-100 тыс. руб.", "100-150 тыс. руб.","150-200 тыс. руб.","свыше 200 тыс. руб."])
total_income_debt = total_income_debt.groupby('income_groups')['debt'].agg(['count', 'sum'])
total_income_debt['%'] = round((total_income_debt[('sum')] / total_income_debt[('count')])*100,2)
total_income_debt.columns = ['Общее количество заемщиков','Количество просрочников','%']
total_income_debt

,Общее количество заемщиков,Количество просрочников,%
income_groups,,,
до 50 тыс. руб.,372,23,6.18
50-100 тыс. руб.,4091,331,8.09
100-150 тыс. руб.,7160,624,8.72
150-200 тыс. руб.,4764,405,8.50
свыше 200 тыс. руб.,5067,358,7.07


### Вывод ###
Прямой зависимости между уровнем дохода и наличием просроченной задолженности нет. Доля просроченных заемщиков колеблется от 8,09% до 8,72%.

## Шаг 4. Общий вывод ##

In [41]:
print("Средняя просрочка по кредитному портфелю: %.1f" % ((df['debt'].sum() / len(df)*100)), '%')

Средняя просрочка по кредитному портфелю: 8.1 %


Для снижения количества просроченных кредитов рекомендуется пересмотреть скоринговую политику в отношении следующих групп клиентов:

* Клиентов с детьми.
* Не женатых/холостых, а также клиентов, состояющих в гражданском браке.
* Клиентов, подающих заявку на автокредитование.